# List Generator

This python notebook will look through corpuses that exist in music21 and save to the folder `roman-numerals` a text file with relevant information to chord phrase frequency analysis.

## Text Files Structure

These text files will consist of a list of roman numerals for all the pieces that have an existing `.rntxt` file under the same composer.

* The characters `,` will separate data for different roman numerals in the same piece
* The characters `\n` will separate data for different pieces by the same composer


## Table of Contents:

* [Method 1 - Record Only Relative Order](#method-1)
    * [a. Listing One Song in Proper Format](#method-1-part-a)
    * [b. Listing Multiple Songs in Proper Format](#method-1-part-b)
    * [c. Create the File for One Composer](#method-1-part-c)
    * [d. Create Files for Several Composers](#method-1-part-d)
* [Method 2 - Record Relative Order and Duration](#method-2)

# Method 1 - Record Only Relative Order <a class="anchor" id="method-1"></a>

In this first approach, I will be importing various corpuses from the existing music21 library which already have the roman text in them and save them to separate text files. As the title would imply, these text files will only record the relative order of these roman numerals for simplicity, and nothing more. This means we will be disregarding parts of the corpus that we may or may not wish to take into account, including duration, root note, and key changes. 

In [1]:
from music21 import corpus, stream

## a. Listing One Song in Proper Format <a class="anchor" id="method-1-part-a"></a>

To begin with, I will load up a single song and generate a string with all of the roman numerals in the song. This song is selected from the music21 module reference page for the `romanText.translate`.

In [2]:
# http://web.mit.edu/music21/doc/moduleReference/moduleRomanTextTranslate.html
monteverdi = corpus.parse('monteverdi/madrigal.3.1.rntxt')

I loop through all elements with the "RomanNumeral" class and get the string that represents the roman numeral. (I currently don't know whether this would get inversions as well). It then joins all of the roman numerals with a comma, and prints. This gives a good idea of what one song will look like encoded using our idea.

In [3]:
monteverdi_chords = []
for part in monteverdi.parts:
    for roman_numerals in part.flat.getElementsByClass('RomanNumeral'):
        monteverdi_chords.append(roman_numerals.romanNumeralAlone)
print(",".join(monteverdi_chords))

vi,V,I,IV,I,V,V,i,V,i,VI,V,i,V,i,i,i,I,IV,ii,vii,I,vi,I,I,IV,vii,I,vi,vi,I,IV,IV,I,I,IV,ii,V,I,V,V,ii,ii,vi,IV,V,I,I,V,V,ii,VI,VI,iv,V,V,i,V,I,V,IV,I,vi,V,I,I,i,i,VII,VII,iv,VI,i,V,i,ii,V,I,vi,V,vi,V,vi,i,V,I,V,ii,V,I,V,vi,i,i,V,i,V,I,i,V,i,V,i,V,I,ii,I,ii,V,I,I,vi,V,II,V,I,IV,ii,vii,V,i,V,I,i,V,i,V,I,ii,I,ii,V,I,ii,I,ii,V,I,I,vi,vi,II,V,I,vi,V,i,V,I,i,i,i,i,i,ii,V,i,V,i,i,i,v,IV,III,V,i,V,i,i,II,III,v,IV,i,vii,i,i,V,V,V,I,vii,I,I,vii,I,V,V,i,VII,v,i,V,V,i,V,I,I,V,vi,V,IV,ii,V,i,V,i,I,IV,IV,I,ii,I,ii,V,i,V,i,V,vi,ii,V,IV,vii,iii,IV,V,V,i,V,iii,vi,ii,V,I,I,vi,ii,I,IV,V,I,V,I,IV,V,vi,iii,i,V,i,VII,v,iv,III,iv,iv,III,I,ii,v,i,V,I


Next, I will generalize the above algorithm to take in a song, and return a comma separated string

In [4]:
def format_score(s : stream.Score) -> str:
    """
    Given a Score, generates a comma-separated string of the roman numerals in
    order that they appear in the score.
    """
    chord_order = []
    for part in s.parts:
        for roman_numerals in part.flat.getElementsByClass('RomanNumeral'):
            chord_order.append(roman_numerals.romanNumeralAlone)
    return ",".join(chord_order)

print(format_score(monteverdi))

vi,V,I,IV,I,V,V,i,V,i,VI,V,i,V,i,i,i,I,IV,ii,vii,I,vi,I,I,IV,vii,I,vi,vi,I,IV,IV,I,I,IV,ii,V,I,V,V,ii,ii,vi,IV,V,I,I,V,V,ii,VI,VI,iv,V,V,i,V,I,V,IV,I,vi,V,I,I,i,i,VII,VII,iv,VI,i,V,i,ii,V,I,vi,V,vi,V,vi,i,V,I,V,ii,V,I,V,vi,i,i,V,i,V,I,i,V,i,V,i,V,I,ii,I,ii,V,I,I,vi,V,II,V,I,IV,ii,vii,V,i,V,I,i,V,i,V,I,ii,I,ii,V,I,ii,I,ii,V,I,I,vi,vi,II,V,I,vi,V,i,V,I,i,i,i,i,i,ii,V,i,V,i,i,i,v,IV,III,V,i,V,i,i,II,III,v,IV,i,vii,i,i,V,V,V,I,vii,I,I,vii,I,V,V,i,VII,v,i,V,V,i,V,I,I,V,vi,V,IV,ii,V,i,V,i,I,IV,IV,I,ii,I,ii,V,i,V,i,V,vi,ii,V,IV,vii,iii,IV,V,V,i,V,iii,vi,ii,V,I,I,vi,ii,I,IV,V,I,V,I,IV,V,vi,iii,i,V,i,VII,v,iv,III,iv,iv,III,I,ii,v,i,V,I


## (b) Listing Multiple Songs in Proper Format <a class="anchor" id="method-1-part-b"></a>

Now that I've made a method that generates the properly formatted string for a single song, the next goal is to repeat this process for multiple songs under the same composer.

One way we could do this would be to manually find the files in the composer's folder which end in `.rntxt` and loop through the list of paths. However, it would be much more convenient for the method to do this automatically, which I don't know how to do yet so I'll leave it here.

In [ ]:
def format_scores_for_composer (composer="monteverdi"):
    
    pass

## (c) Create the File for One Composer <a class="anchor" id="method-1-part-c"></a>

## (d) Create Files for Several Composers <a class="anchor" id="method-1-part-d"></a>

# Method 2 - Record Relative Order and Duration <a class="anchor" id="method-2"></a>